### Task 2: Determine within which grid cell a point/particle falls into
### Task 3: Interpolate velocity to the particle lat/lon 
### Task 4: Advance forward

- variables u,v (current velocity components)
- use only surface data: depth (z) = -2.0 
- use first time for now [0]

Note: current velocities u, v are on different grids, u(eta_u,xi_u) and v(eta_v, xi_v).
Latitudes and longitudes corresponding values for current are in the coordinate arrayes lat_u, lon_u, lat_v, lon_v


In [1]:
# task1: acquire data
import numpy as np
import pandas as pd
import xarray as xr
import geopy


# url='https://oceanmodeling.ucsc.edu:8443/thredds/dodsC/ccsra_2016a_phys_agg_zlevs/fmrc/CCSRA_2016a_Phys_ROMS_z-level_(depth)_Aggregation_best.ncd'
url='dataset_2010.nc'
actual = xr.open_dataset(url)

# ds = actual.copy()
# ("ds.nc")


lat1 = 38.0
lon1 = -123.0

actual

<xarray.Dataset>
Dimensions:      (z: 11, eta_rho: 181, xi_rho: 186, eta_u: 181, xi_u: 185, eta_v: 180, xi_v: 186, ocean_time: 1, time: 101)
Coordinates:
  * z            (z) float64 -250.0 -200.0 -150.0 -100.0 ... -10.0 -5.0 -2.0
    lat_rho      (eta_rho, xi_rho) float64 ...
    lat_u        (eta_u, xi_u) float64 ...
    lat_v        (eta_v, xi_v) float64 ...
    lon_rho      (eta_rho, xi_rho) float64 ...
    lon_u        (eta_u, xi_u) float64 ...
    lon_v        (eta_v, xi_v) float64 ...
  * ocean_time   (ocean_time) datetime64[ns] 2022-07-09
  * time         (time) datetime64[ns] 2011-01-02 2011-01-03 ... 2011-04-12
    time_run     (time) datetime64[ns] ...
Dimensions without coordinates: eta_rho, xi_rho, eta_u, xi_u, eta_v, xi_v
Data variables: (12/15)
    angle        (time, eta_rho, xi_rho) float64 ...
    h            (time, eta_rho, xi_rho) float64 ...
    mask_rho     (time, eta_rho, xi_rho) float64 ...
    mask_u       (time, eta_u, xi_u) float64 ...
    mask_v       (time, eta_v, xi_v) float64 ...
    ubar         (time, eta_u, xi_u) float64 ...
    ...           ...
    salt         (time, z, eta_rho, xi_rho) float32 ...
    rho          (time, z, eta_rho, xi_rho) float32 ...
    u            (time, z, eta_u, xi_u) float32 ...
    v            (time, z, eta_v, xi_v) float32 ...
    w            (time, z, eta_rho, xi_rho) float32 ...
    time_offset  (time) datetime64[ns] ...
Attributes: (12/46)
    file:                            output/da_fwd_002.nc
    format:                          netCDF-3 64bit offset file
    Conventions:                     CF-1.4
    type:                            ROMS/TOMS history file
    title:                           UCSC California Current System, wc12.0_c...
    state_vector:                    zeta, ubar, vbar, u, v, temp, salt
    ...                              ...
    NCO:                             "4.5.2"
    _CoordSysBuilder:                ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:                   GRID
    featureType:                     GRID
    location:                        Proto fmrc:CCSRA_2016a_Phys_ROMS_z-level...
    DODS_EXTRA.Unlimited_Dimension:  ocean_time

In [2]:
"""
Chunks down and downloads the remote dataset from UCSC's server
"""
# end_index = len(actual.time.values)
# master = actual.sel(time = actual.time.values[0])
# for i in range(1, end_index):
#     temp = actual.sel(
#         time = actual.time.values[i],
#         z = slice(0, 1)
#     )
#     master = xr.concat([master, temp], dim='time')
#     if i % 100 == 0:
#         name = f"backup_{i}.nc"
#         master.to_netcdf(name)
#     print(i)
# master.to_netcdf("ds.nc")
# t = xr.open_dataset("ds.nc")
# t

"\nChunks down and downloads the remote dataset from UCSC's server\n"

In [ ]:
# Task2: find nearest grid points for current components u and v.

for x in range(len(actual.lon_u)):
    print(actual.lon_u[0, x].lon_u.values)

In [3]:
# Task3: interpolate velocity (u,v) to lat1 and lat2 (this point might not fall on a grid point)

actual = actual.interp(eta_u=actual.eta_rho, xi_u=actual.xi_rho, method='linear')
actual = actual.interp(eta_v=actual.eta_rho, xi_v=actual.xi_rho, method='linear')

In [4]:
# drop the lat_rho and lon_rho categories
# actual = actual.drop(['lat_rho', 'lon_rho'])
try:
    actual = actual.drop(['lat_u', 'lon_u', 'lat_v', 'lon_v'])
except ValueError:
    print("coords already removed")
print(f" length of thing {len(actual.lat_rho)}")
actual


 length of thing 181


<xarray.Dataset>
Dimensions:      (z: 11, eta_rho: 181, xi_rho: 186, ocean_time: 1, time: 101)
Coordinates:
  * z            (z) float64 -250.0 -200.0 -150.0 -100.0 ... -10.0 -5.0 -2.0
    lat_rho      (eta_rho, xi_rho) float64 30.0 30.0 30.0 ... 48.0 48.0 48.0
    lon_rho      (eta_rho, xi_rho) float64 -134.0 -133.9 ... -115.6 -115.5
  * ocean_time   (ocean_time) datetime64[ns] 2022-07-09
  * time         (time) datetime64[ns] 2011-01-02 2011-01-03 ... 2011-04-12
    time_run     (time) datetime64[ns] 2011-01-02 2011-01-03 ... 2011-04-12
Dimensions without coordinates: eta_rho, xi_rho
Data variables: (12/19)
    angle        (time, eta_rho, xi_rho) float64 0.0004363 ... 0.0006485
    h            (time, eta_rho, xi_rho) float64 4.497e+03 4.497e+03 ... 10.0
    mask_rho     (time, eta_rho, xi_rho) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    mask_u       (time, eta_rho, xi_rho) float64 1.0 1.0 1.0 1.0 ... 0.0 0.0 nan
    mask_v       (time, eta_rho, xi_rho) float64 1.0 1.0 1.0 1.0 ... nan nan nan
    ubar         (time, eta_rho, xi_rho) float64 0.01585 0.01274 ... nan nan
    ...           ...
    w            (time, z, eta_rho, xi_rho) float32 -0.001627 -0.001627 ... nan
    time_offset  (time) datetime64[ns] 2011-01-02 2011-01-02 ... 2011-01-02
    eta_u        (eta_rho) int64 0 1 2 3 4 5 6 7 ... 174 175 176 177 178 179 180
    xi_u         (xi_rho) int64 0 1 2 3 4 5 6 7 ... 179 180 181 182 183 184 185
    eta_v        (eta_rho) int64 0 1 2 3 4 5 6 7 ... 174 175 176 177 178 179 180
    xi_v         (xi_rho) int64 0 1 2 3 4 5 6 7 ... 179 180 181 182 183 184 185
Attributes: (12/46)
    file:                            output/da_fwd_002.nc
    format:                          netCDF-3 64bit offset file
    Conventions:                     CF-1.4
    type:                            ROMS/TOMS history file
    title:                           UCSC California Current System, wc12.0_c...
    state_vector:                    zeta, ubar, vbar, u, v, temp, salt
    ...                              ...
    NCO:                             "4.5.2"
    _CoordSysBuilder:                ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:                   GRID
    featureType:                     GRID
    location:                        Proto fmrc:CCSRA_2016a_Phys_ROMS_z-level...
    DODS_EXTRA.Unlimited_Dimension:  ocean_time

In [5]:
# Will or austin do this
def change_in_meter_to_change_in_degrees(lat, lon, delta_lat_m, delta_lon_m):

    # lat: starting point for flatitude (Float)
    # lon: starting point for longitude (Float)
    # delta_lat_m: change in meters on the lat 'axis' (Float)
    # delta_lon_m: change in meters on the lon 'axis' (Float)
    # returns: (new_lat, new_lon) a tuple containing the new coords (both floats)

    # meters to degrees
    
    start = geopy.Point(lat, lon)
    distance = geopy.distance.VincentyDistance(meters = delta_lat_m)
    new_lat = distance.destination(point = start, bearing = 0).latitude
    distance = geopy.distance.VincentyDistance(meters = delta_lon_m)
    new_lon = distance.destination(point = start, bearing = 0).longitude
    
    return (new_lat, new_lon)

# I will do this one
def get_indices_from_coords(lat, lon):
    # lat: flatitude to find indices for
    # lon: longitude to find indices for
    # returns: the same point, but as an index (like with decimals n' stuff)
    final_lat = None
    global actual
    for i, temp_lat in enumerate(actual["lat_rho"]):
        if temp_lat>lat:
            prev_lat = actual["lat_rho"][i-1]
            final_lat = i+((lat-prev_lat)/temp_lat-prev_lat)
            break
    if final_lat is None:
        raise Exception("Latitude not in range of dataset")
    for i, temp_lon in enumerate(actual["lon_rho"]):
        if temp_lon>lon:
            prev_lon = actual["lon_rho"][i-1]
            final_lon = i+((lon-prev_lon)/temp_lon-prev_lon)
            tuple_out = (final_lat, final_lon)
            return tuple_out
    raise Exception("Longitude not in range of dataset")


# I may do this one later but feel free
def interpolate_uv(lat_rho, lon_rho):
    # lat_rho: the index on the flatitude 'axis' to find u and v (FLOAT, not an integer)
    # lon_rho: the index on the longitude 'axis' to find u and v (FLOAT, not an integer)
    # return: velocity at point
    # you will need to interpolate U and V to find its value at the given coords
    global actual
    tuple_out = (1, 1)
    return tuple_out

In [12]:
# Task4: move particle forward 
# we need to calculate the position of the particle if it moves for one time step (a day in this data set) with velocity u, v
# there are different methods to implement that, we will use the Runge-Kutta order 4. but you can start w an smipl

class Point:

    def __init__(self, lat, lon, time):
        self.lat = lat # float
        self.lon = lon # float
        self.time = time
        self.point_history = [] # used for animating point progress
    def do_movement_tick(self):



        # U and V are in m/s,
        index_pair = get_indices_from_coords(self.lat, self.lon)
        # TODO: find interpolated u and v from indices
        uv_pair = interpolate_uv(index_pair[0], index_pair[1])
        final_coord_pair = change_in_meter_to_change_in_degrees(self.lat, self.lon, uv_pair[0], uv_pair[1])


        self.lat = final_coord_pair[0]
        self.lon = final_coord_pair[1]
        self.time += 1

        self.point_history.append(self.lat, self.lon, self.time)
    def __str__(self):
        return f"({self.x}, {self.y})"

    # def

# Task5: plot the particle path
# we need to plot the path of the particle

# pt = Point(x=lon1, y=lat1, time=5)
# print(pt)
#
# # move the point
# pt.move(u=actual.u[0, 0, 0, 0].values, v=actual.v[0, 0, 0, 0].values, dt=1)
# print(pt)

(38.0, -123.0)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [7]:
# Task4: move particle forward 
# we need to calculate the position of the particle if it moves for one time step (a day in this data set) with velocity u, v
# there are different methods to implement that, we will use the Runge-Kutta order 4. but you can start w an smipl

class point:
    def __init__(self, x, y, time):
        self.x = x
        self.y = y
        self.time = time
    def move(self, u, v, dt):
        # TODO: Do the thing
        self.x += u * dt
        self.y += v * dt
    def __str__(self):
        return f"({self.x}, {self.y})"

# Task5: plot the particle path
# we need to plot the path of the particle

pt = point(x=lon1, y=lat1, time=5)
print(pt)

# move the point
pt.move(u=actual.u[0, 0, 0, 0].values, v=actual.v[0, 0, 0, 0].values, dt=1)
print(pt)

pt.do

(-123.0, 38.0)
(-122.97695616073906, 37.89688344299793)
